In [2]:
%reload_ext autoreload
%autoreload 2
from simulation_v3 import Sim
from simulation_v3 import SimTester

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import style
import pandas as pd
import os
import sys
import time


In [9]:
# In matrix form using the new method:

# define the network adjacency matrix
    # Servers/Sources/Queues defined on diagional as:
    # 1 on diagonal: customer arrives
    # -1 on diagonal: server with service time
    # 0 on diagonal: server with no service time
        # customer can only move to a server if not busy

    # Connections between elements of system defined off diagonal as:
    # sum(non-diagonal) = 1:
        # entries are the probability of a customer moving from one element to another
    # sum(non-diagonal) > 1 (ie, each child node has 1 in adj matrix):
        # send customers to all child node with shortest queue
    # sum(non-diagonal) = 0:
        # server leads to a sink

adj_sim    = np.array([ [ 1,  1,  0,  0,  0,  0,  0,  0 ],
                        [ 0, -0.33,  0.75,  0,  0,  0.25,  0,  0 ],
                        [ 0,  0, -.58,  0,  0,  0,  0,  0 ],
                        [ 0,  0,  0.5,  1,  0,  0.5,  0,  0 ],
                        [ 0,  0,  0,  0, -0.79,  0,  0.7,  0.3 ], 
                        [ 0,  0.5,  0,  0,  0.5, -0.24,  0,  0 ], 
                        [ 0,  0,  0,  0,  0, 0,  -1,  0 ],
                        [ 0,  0,  0,  0,  0.8, 0,   0.2,  -0.9 ],])

# max queue lengths
queue_list = [5,3,7,1,8,4,21,32]

probabilities = [0.2, 0.7, 0.1]
values = [0.5, 0.9, 0.83]

# define the distributions for each server
    # name of distribution, parameters (example: ['exponential', mean=1.0])
distributions = [['exponential', 1.764705882352],
                 ['normal', 0.75, 0.5],
                 ['discrete', values, probabilities],
                 ['discrete', values, probabilities],
                 ['discrete', values, probabilities],
                 ['exponential', 1.764705882352],
                 ['beta', 0.25, 5, 0.1, 0.9],
                 ['gamma', 0.5, 3, 1.1],
                 ['normal', 0.67, 2]]

np.random.seed(42)
seeds = np.random.randint(0, 99999, size=1)
sim = Sim(adj_sim, distributions, queue_list, seeds=seeds, generate_log=True, animation=False, record_history=False, logging_mode='Music')

sim.run(number_of_customers=1000)

1: 0.9909970760345459 elapsed time for 559.3199999999947 simulation time with 1000 customers


In [11]:
%reload_ext autoreload
%autoreload 2

from sim_log_process_music import process_adjsim_log

# this file is still very rough.... The note levels are being randomly selected 30-127

# velcoity are based off queue length

# many more tuning improvements to be made - Change the matrix above for highly variable output. 

process_adjsim_log()

In [12]:
import mido
from mido import MidiFile
import pygame

def play_midi(file_path, print_midi=False):
    midi_file = MidiFile(file_path)

    if print_midi:
        for i, track in enumerate(midi_file.tracks):
            print('Track {}: {}'.format(i, track.name))
            for msg in track:
                print(msg)

    pygame.init()
    pygame.mixer.music.load(file_path)


    # pygame change volume
    pygame.mixer.music.set_volume(5)
    pygame.mixer.music.play()


play_midi('output.mid', print_midi=False)

pygame 2.5.2 (SDL 2.28.3, Python 3.9.16)
Hello from the pygame community. https://www.pygame.org/contribute.html
